pip install pyaudio

In [1]:
from sys import byteorder
from array import array
from struct import pack
import time
import threading
from queue import Queue

import pyaudio
import wave
import os

THRESHOLD = 500
CHUNK_SIZE = 1024
FORMAT = pyaudio.paInt16
RATE = 8000
RECORD_SECONDS = 3
WAVE_OUTPUT_FILENAME_EXTENSION = 0
WAVE_OUTPUT_FILENAME = "output"

q = Queue()
lock = threading.Lock()

def is_silent(snd_data):
    "Returns 'True' if below the 'silent' threshold"
    return max(snd_data) < THRESHOLD

def normalize(snd_data):
    "Average the volume out"
    MAXIMUM = 16384
    times = float(MAXIMUM)/max(abs(i) for i in snd_data)

    r = array('h')
    for i in snd_data:
        r.append(int(i*times))
    return r

def trim(snd_data):
    "Trim the blank spots at the start and end"
    def _trim(snd_data):
        snd_started = False
        r = array('h')

        for i in snd_data:
            if not snd_started and abs(i)>THRESHOLD:
                snd_started = True
                r.append(i)

            elif snd_started:
                r.append(i)
        return r

    # Trim to the left
    snd_data = _trim(snd_data)

    # Trim to the right
    snd_data.reverse()
    snd_data = _trim(snd_data)
    snd_data.reverse()
    return snd_data

def add_silence(snd_data, seconds):
    "Add silence to the start and end of 'snd_data' of length 'seconds' (float)"
    r = array('h', [0 for i in range(int(seconds*RATE))])
    r.extend(snd_data)
    r.extend([0 for i in range(int(seconds*RATE))])
    return r

def record():
    """
    Record a word or words from the microphone and 
    return the data as an array of signed shorts.

    Normalizes the audio, trims silence from the 
    start and end.
    """
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=1, rate=RATE, input=True, output=True, frames_per_buffer=CHUNK_SIZE)

    num_silent = 0
    snd_started = False

    r = array('h')
    count = 0
#     print()
    for i in range(0, int(RATE / CHUNK_SIZE * RECORD_SECONDS)):
        count +=1
        # little endian, signed short
        snd_data = array('h', stream.read(CHUNK_SIZE))
        if byteorder == 'big':
            snd_data.byteswap()
        r.extend(snd_data)

        #print('\r%08d' % count)
        silent = is_silent(snd_data)

        if silent and snd_started:
            num_silent += 1
        elif not silent and not snd_started:
            snd_started = True

    sample_width = p.get_sample_size(FORMAT)
    stream.stop_stream()
    stream.close()
    p.terminate()

    r = normalize(r)
    r = trim(r)
#     r = add_silence(r, 0.5)
    return sample_width, r

def record_to_file():
    with lock:
        WAVE_OUTPUT_FILENAME_EXTENSION = 0
        WAVE_OUTPUT_FILENAME = "rec"
        d= True
        
        ts = time.time()
        EXPORT_FOLDER = "RECORDINGS_" + str(ts).split(".")[0]
        if not os.path.exists(EXPORT_FOLDER):
            os.makedirs(EXPORT_FOLDER)
            
        while d:
            q.join()
            print('Recording')
            sample_width, data = record()
            data = pack('<' + ('h'*len(data)), *data)
            print('Saving........')

            wf = wave.open(EXPORT_FOLDER + "/" + WAVE_OUTPUT_FILENAME + "_" + str(WAVE_OUTPUT_FILENAME_EXTENSION) + ".wav", 'wb')
            wf.setnchannels(1)
            wf.setsampwidth(sample_width)
            wf.setframerate(RATE)
            wf.writeframes(data)
            wf.close()

            WAVE_OUTPUT_FILENAME_EXTENSION += 1
            
#             Exit the loop with enter
            try:
                if q.get(timeout = 0) == 1:
                    d = False
            except:
                pass
        
        print('Task Completing')
        q.task_done()

if __name__ == '__main__':
    print('Press Enter to stop Recording')
    t = threading.Thread(target=record_to_file)
    t.daemon=True
    t.start()
    input()
    q.put(1)
    q.join()
    print('Finished')

RecordingYou're live! Press Enter to finish.

Saving........
Recording
Saving........
Recording
Saving........
Recording

Saving........
FinishedTask Completing

